In [1]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
import numpy as np

In [2]:
# DATASET DIRECTORY CONFIGURATION

train_dir = "train"
test_dir = "test"

In [3]:
# IMAGE PARAMETERS

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [4]:
# DATA PREPROCESSING & AUGMENTATION

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1183 images belonging to 2 classes.


In [5]:
# IMPROVED CNN MODEL ARCHITECTURE

initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Improved model with dropout and regularization
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.3),
    
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.4),
    
    layers.Flatten(),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Configure the model optimizers, loss function, and metrics
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

c:\Users\Justin\25-26\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# TRAINING THE CNN

history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator
)

Epoch 1/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 108s 889ms/step - accuracy: 0.5322 - loss: 1.0616 - val_accuracy: 0.5407 - val_loss: 0.8289
Epoch 2/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 105s 866ms/step - accuracy: 0.6650 - loss: 0.6986 - val_accuracy: 0.6772 - val_loss: 0.6237
Epoch 3/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 103s 865ms/step - accuracy: 0.7616 - loss: 0.5646 - val_accuracy: 0.7291 - val_loss: 0.5886
Epoch 4/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 102s 853ms/step - accuracy: 0.8091 - loss: 0.4785 - val_accuracy: 0.8656 - val_loss: 0.3918
Epoch 5/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 99s 829ms/step - accuracy: 0.8139 - loss: 0.4682 - val_accuracy: 0.8762 - val_loss: 0.4252
Epoch 6/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 100s 835ms/step - accuracy: 0.8097 - loss: 0.4490 - val_accuracy: 0.8772 - val_loss: 0.3862
Epoch 7/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 99s 828ms/step - accuracy: 0.8387 - loss: 0.4095 - val_accuracy: 0.8381 - val_loss: 0.4260
Epoch 8/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 98s 822ms/step - accuracy: 0.8398 - lo

In [7]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 11s 299ms/step - accuracy: 0.8157 - loss: 0.4066
Test Accuracy: 0.8157227635383606


In [8]:
# SAVE THE MODEL
model.save('muffin_vs_chihuahua_cnn.h5')

In [9]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='muffin_vs_chihuahua_cnn.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    confidence = pred if pred >= 0.5 else 1 - pred
    print(f"Prediction: {label} (confidence: {confidence:.2%})")
    return label, confidence

In [10]:
# Example usage:
predict_image("run/run_1.jpg")
predict_image("run/run_2.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
Prediction: Muffin (confidence: 75.53%)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
Prediction: Chihuahua (confidence: 99.86%)


('Chihuahua', np.float32(0.9985993))